In [1]:
from rosemary import jpt_parse_args, jpt_setup; jpt_setup()

import sys
sys.path.append('/dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/')

/dccstor/mit_fm/wpq/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [2]:
import argparse
import os
import tqdm
import re
import json
import random
import time
import evaluate
from transformers import GPT2LMHeadModel
from eval.utils import generate_completions, load_hf_lm_and_tokenizer, query_openai_chat_model, KeyWordsCriteria
from eval.gsm.examplars import EXAMPLARS as GSM_EXAMPLARS
import torch

In [3]:

parser = argparse.ArgumentParser()
parser.add_argument("--data_dir", type=str, default="data/mgsm")
parser.add_argument("--max_num_examples", type=int, default=None, help="maximum number of examples to evaluate.")
parser.add_argument("--save_dir", type=str, default="results/mgsm")
parser.add_argument("--model_name_or_path", type=str, default=None, help="if specified, we will load the model to generate the predictions.")
parser.add_argument("--tokenizer_name_or_path", type=str, default=None, help="if specified, we will load the tokenizer from here.")
parser.add_argument("--openai_engine", type=str, default=None, help="if specified, we will use the OpenAI API to generate the predictions.")
parser.add_argument("--n_shot", type=int, default=8, help="max number of examples to use for demonstration.")
parser.add_argument("--no_cot", action="store_true", help="If given, we're evaluating a model without chain-of-thought.")
parser.add_argument("--eval_batch_size", type=int, default=1, help="batch size for evaluation.")
parser.add_argument("--load_in_8bit", action="store_true", help="load model in 8bit mode, which will reduce memory and speed up inference.")
parser.add_argument("--gptq", action="store_true", help="If given, we're evaluating a 4-bit quantized GPTQ model.")
parser.add_argument("--use_chat_format", action="store_true", help="If given, the prompt will be encoded as a chat format with the roles in prompt.")

# model_name_or_path = 'gpt2-Large'
# model_name_or_path = '../results/gpt2-Large_human_mix'
# model_name_or_path = 't5-Large'
# model_name_or_path = 'google/flan-t5-large'
# model_name_or_path = '../results/google/flan-t5-small'
# model_name_or_path = 'huggyllama/llama-7b'
# model_name_or_path = '../results/baselines/mosaicml/mpt-7b'
# model_name_or_path = '../results/baselines/mosaicml/mpt-7b'
# model_name_or_path = '../results/baselines/t5-11b'
model_name_or_path = '../results/huggyllama:llama-7b_human_mix-trainer_savebystep/checkpoint-200'

cmd = f"""
    --data_dir ../data/eval/gsm/ \
    --save_dir {model_name_or_path}/eval/gsm/ \
    --max_num_examples 20 \
    --model_name_or_path {model_name_or_path} \
    --eval_batch_size 5 \
    --n_shot 8
"""

args = jpt_parse_args(parser, cmd)

# model_name_or_path and openai_engine cannot be both None or both not None.
assert (args.model_name_or_path is None) != (args.openai_engine is None), "Either model_name_or_path or openai_engine should be specified."
args


Namespace(data_dir='../data/eval/gsm/', max_num_examples=20, save_dir='../results/huggyllama:llama-7b_human_mix-trainer_savebystep/checkpoint-200/eval/gsm/', model_name_or_path='../results/huggyllama:llama-7b_human_mix-trainer_savebystep/checkpoint-200', tokenizer_name_or_path=None, openai_engine=None, n_shot=8, no_cot=False, eval_batch_size=5, load_in_8bit=False, gptq=False, use_chat_format=False)

In [4]:
exact_match = evaluate.load("exact_match")

random.seed(42)

print("Loading data...")
test_data = []
with open(os.path.join(args.data_dir, f"test.jsonl")) as fin:
    for line in fin:
        example = json.loads(line)
        test_data.append({
            "question": example["question"],
            "answer": example["answer"].split("####")[1].strip()
        })

# some numbers are in the `x,xxx` format, and we want to remove the comma
for example in test_data:
    example["answer"] = re.sub(r"(\d),(\d)", r"\1\2", example["answer"])
    assert float(example["answer"]), f"answer is not a valid number: {example['answer']}"
    

if args.max_num_examples and len(test_data) > args.max_num_examples:
    test_data = random.sample(test_data, args.max_num_examples)


Loading data...


In [5]:
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir, exist_ok=True)
    
global GSM_EXAMPLARS
if args.n_shot:
    if len(GSM_EXAMPLARS) > args.n_shot:
        GSM_EXAMPLARS = random.sample(GSM_EXAMPLARS, args.n_shot)
    demonstrations = []
    for example in GSM_EXAMPLARS:
        if args.no_cot:
            demonstrations.append(
                "Quesion: " + example["question"] + "\n" + "Answer: " + example["short_answer"]
            )
        else:
            demonstrations.append(
                "Question: " + example["question"] + "\n" + "Answer: " + example["cot_answer"]
            )
    prompt_prefix = "Answer the following questions.\n\n" + "\n\n".join(demonstrations) + "\n\n"
else:
    prompt_prefix = "Answer the following question.\n\n"

prompts = []
for example in test_data:
    if args.use_chat_format:
        prompt = "<|user|>\n" + prompt_prefix + "Question: " + example["question"].strip() + "\n<|assistant|>\n" + "Answer:"
    else:
        prompt = prompt_prefix + "Question: " + example["question"].strip() + "\nAnswer:"
    prompts.append(prompt)
    
print(prompts[0])

Answer the following questions.

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Answer: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. So the answer is 6.

Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Answer: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. So the answer is 5.

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
Answer: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. So the answer is 39.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
Answer: Jason 

In [6]:
# model_name_or_path = '../'+args.model_name_or_path
# tokenizer_name_or_path = None
# load_in_8bit = args.load_in_8bit
# dtype = torch.bfloat16
# gptq_model = args.gptq
# use_fast_tokenizer = True
# padding_side = 'left'
# device_map = 'auto'


# checkpoint_name = os.path.join(model_name_or_path, 'adapter_model.bin')
# if os.path.exists(checkpoint_name):
#     from peft import PeftModel, PeftConfig
#     peft_model_name_or_path = model_name_or_path
#     config = PeftConfig.from_pretrained(peft_model_name_or_path)
#     model, tokenizer = load_hf_lm_and_tokenizer(
#         model_name_or_path=config.base_model_name_or_path, 
#         tokenizer_name_or_path=config.base_model_name_or_path,
#         device_map=device_map,
#         gptq_model=gptq_model,
#         load_in_8bit=load_in_8bit,
#         dtype=dtype,
#         use_fast_tokenizer=use_fast_tokenizer,
#         padding_side=padding_side,
#     )
#     peft_model = PeftModel.from_pretrained(model, peft_model_name_or_path)
#     model = peft_model.base_model.merge_and_unload() # merge LoRA weights to base model weights.
#     embedding_size = model.get_input_embeddings().weight.shape[0]
#     if len(tokenizer) > embedding_size:
#         print(f"The vocabulary size of the tokenizer in the LoRA model folder"
#               f"contains {len(tokenizer)-embedding_size} more tokens than the base model.\n"
#                "Resizing the token embeddings of the merged model...")
#         model.resize_token_embeddings(len(tokenizer))

In [7]:
from eval.utils import load_hf_lm_and_tokenizer

model, tokenizer = load_hf_lm_and_tokenizer(
    model_name_or_path=args.model_name_or_path, 
    tokenizer_name_or_path=args.tokenizer_name_or_path, 
    load_in_8bit=args.load_in_8bit,
    dtype=torch.bfloat16,
    gptq_model=args.gptq,
    use_fast_tokenizer=True,
    device_map='auto',
)

model.device, model.dtype


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 120
CUDA SETUP: Loading binary /dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda120.so...


/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /dccstor/mit_fm/miniconda/envs/open-instruct did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/opt/ibm/lsfsuite/ext/ppm/10.2/linux2.6-glibc2.3-x86_64/lib')}
  warn(msg)
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /opt/ibm/lsfsuite/lsf/10.1/linux2.6-glibc2.3-x86_64/lib:/opt/ibm/lsfsuite/ext/ppm/10.2/linux2.6-glibc2.3-x86_64/lib did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING:

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [41]:
from eval.utils import generate_completions


# get the last token because the tokenizer may add space tokens at the start.
# wpq: t5 tokenizer strips `\n`. don't use `\n` as stop sequence. just generate to max length or encounters <\s>. 
new_line_token = tokenizer.encode("\n", add_special_tokens=False)
stop_id_sequences = [[new_line_token[-1]]] if new_line_token else None

if isinstance(model, GPT2LMHeadModel):
    # wpq: for gpt-2 model, need to enforce `max_length` constraints to avoid `position_id` index errors.
    generation_kwargs = {'max_length': model.config.max_position_embeddings} # 1024
else:
    # wpq: modify `max_new_tokens=512` to `256` by default
    generation_kwargs = {'max_new_tokens': 256}


t0 = time.time()
outputs = generate_completions(
    model=model,
    tokenizer=tokenizer,
    prompts=prompts,
    batch_size=args.eval_batch_size,
    stop_id_sequences=stop_id_sequences,
    **generation_kwargs,
)

t = time.time()-t0
print(f'Time = {t:.2f}')
outputs[0]


# batch_siz = 20
# 4*60+44 / 20 = 14.2 / data
# 276.43 / 20 = 13.8 / data

Generating Completions: 100%|██████████| 20/20 [04:58<00:00, 14.95s/it]

Time = 298.92


' ['

In [50]:
## utils.generate_completions*


# if 'gpt2' in model_name_or_path:
#     generation_kwargs = {'max_length': model.config.max_position_embeddings} # 1024
# else:
#     generation_kwargs = {'max_new_tokens': 512}
# print(generation_kwargs)
    
# batch_size = 5
# i = 0
# batch_prompts = prompts[i:i+batch_size]
# # batch_prompts = [
# #     'Is the following sentence positive or negative: I hate the food',
# #     'translate from english to german: the weather is great!']
# tokenized_prompts = tokenizer(batch_prompts, padding="longest", return_tensors="pt", add_special_tokens=False)
# batch_input_ids = tokenized_prompts.input_ids
# attention_mask = tokenized_prompts.attention_mask

# print(batch_input_ids.shape, batch_input_ids.device)
# print(attention_mask.shape, attention_mask.device)
# batch_input_ids = batch_input_ids.cuda()
# attention_mask = attention_mask.cuda()
# print(model.device)


# from transformers import StoppingCriteriaList
# import time
# start = time.time()

# stopping_criteria = StoppingCriteriaList([KeyWordsCriteria(stop_id_sequences)]) if stop_id_sequences else None

# batch_outputs = model.generate(
#     input_ids=batch_input_ids,
#     attention_mask=attention_mask,
#     stopping_criteria=stopping_criteria,
#     **generation_kwargs,
# )
# end = time.time()
# end-start
# print(end-start)
# print(batch_outputs.shape)

# if stop_id_sequences:
#     for output_idx in range(batch_outputs.shape[0]):
#         for token_idx in range(batch_input_ids.shape[1], batch_outputs.shape[1]):
#             if any(batch_outputs[output_idx, token_idx: token_idx+len(stop_sequence)].tolist() == stop_sequence for stop_sequence in stop_id_sequences):
#                 batch_outputs[output_idx, token_idx:] = tokenizer.pad_token_id
#                 break


# # 
# # 15s
# # 18.369792938232422

# num_return_sequences = 1
# # remove the prompt from the output
# # we need to re-encode the prompt because we need to make sure the special tokens are treated the same way as in the outputs.
# # we changed our previous way of truncating the output token ids dicrectly because some tokenizer (e.g., llama) won't add space token before the first token.
# # space is important for some tasks (e.g., code completion).
# batch_outputs = tokenizer.batch_decode(batch_outputs, skip_special_tokens=True)
# batch_prompts = tokenizer.batch_decode(batch_input_ids, skip_special_tokens=True)
# # duplicate the prompts to match the number of return sequences
# batch_prompts = [prompt for prompt in batch_prompts for _ in range(num_return_sequences)]
# batch_generations = [
#     output[len(prompt):] for prompt, output in zip(batch_prompts, batch_outputs)
# ]

# print([len(x) for x in batch_outputs])
# print([len(x) for x in batch_prompts])

{'max_new_tokens': 512}
torch.Size([5, 689]) cpu
torch.Size([5, 689]) cpu
cuda:0
16.02927827835083
torch.Size([5, 115])


In [40]:
# num_return_sequences = 1
# batch_outputs = tokenizer.batch_decode(batch_outputs, skip_special_tokens=True)
# batch_prompts = tokenizer.batch_decode(batch_input_ids, skip_special_tokens=True)
# # duplicate the prompts to match the number of return sequences
# batch_prompts = [prompt for prompt in batch_prompts for _ in range(num_return_sequences)]
# batch_generations = [
#     output[len(prompt):] for prompt, output in zip(batch_prompts, batch_outputs)
# ]
# batch_generations

NameError: name 'batch_outputs' is not defined

In [27]:
# generation_kwargs = {'max_new_tokens': 512}
# disable_tqdm = False
# stop_id_sequences = [[new_line_token]]
# batch_size=args.eval_batch_size



# generations = []
# if not disable_tqdm:
#     progress = tqdm.tqdm(total=len(prompts), desc="Generating Completions")

# num_return_sequences = generation_kwargs.get("num_return_sequences", 1)
# for i in range(0, len(prompts), batch_size):
#     batch_prompts = prompts[i:i+batch_size]
#     tokenized_prompts = tokenizer(batch_prompts, padding="longest", return_tensors="pt", add_special_tokens=False)
#     batch_input_ids = tokenized_prompts.input_ids
#     attention_mask = tokenized_prompts.attention_mask

#     if model.device.type == "cuda":
#         print(torch.cuda.is_available())
#         print(batch_input_ids.device, batch_input_ids.shape)
#         batch_input_ids = batch_input_ids.cuda()
#         attention_mask = attention_mask.cuda()

#     try:
#         batch_outputs = model.generate(
#             input_ids=batch_input_ids,
#             attention_mask=attention_mask,
#             stopping_criteria=[KeyWordsCriteria(stop_id_sequences)] if stop_id_sequences else None,
#             **generation_kwargs
#         )

#         # the stopping criteria is applied at batch level, so if other examples are not stopped, the entire batch will continue to generate.
#         # so some outputs still have the stop sequence, which we need to remove.
#         if stop_id_sequences:
#             for output_idx in range(batch_outputs.shape[0]):
#                 for token_idx in range(batch_input_ids.shape[1], batch_outputs.shape[1]):
#                     if any(batch_outputs[output_idx, token_idx: token_idx+len(stop_sequence)].tolist() == stop_sequence for stop_sequence in stop_id_sequences):
#                         batch_outputs[output_idx, token_idx:] = tokenizer.pad_token_id
#                         break

#         # remove the prompt from the output
#         # we need to re-encode the prompt because we need to make sure the special tokens are treated the same way as in the outputs.
#         # we changed our previous way of truncating the output token ids dicrectly because some tokenizer (e.g., llama) won't add space token before the first token.
#         # space is important for some tasks (e.g., code completion).
#         batch_outputs = tokenizer.batch_decode(batch_outputs, skip_special_tokens=True)
#         batch_prompts = tokenizer.batch_decode(batch_input_ids, skip_special_tokens=True)
#         # duplicate the prompts to match the number of return sequences
#         batch_prompts = [prompt for prompt in batch_prompts for _ in range(num_return_sequences)]
#         batch_generations = [
#             output[len(prompt):] for prompt, output in zip(batch_prompts, batch_outputs)
#         ]
#     except Exception as e:
#         print("Error when generating completions for batch:")
#         print(batch_prompts)
#         print("Error message:")
#         print(e)
#         print("Use empty string as the completion.")
#         batch_generations = [""] * len(batch_prompts) * num_return_sequences

#     generations += batch_generations

#     # for prompt, generation in zip(batch_prompts, batch_generations):
#     #     print("========")
#     #     print(prompt)
#     #     print("--------")
#     #     print(generation)

#     if not disable_tqdm:
#         progress.update(len(batch_prompts)//num_return_sequences)

In [42]:

predictions = []
for output in outputs:
    # replace numbers like `x,xxx` with `xxxx`
    output = re.sub(r"(\d),(\d)", r"\1\2", output)
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", output)
    if numbers:
        predictions.append(numbers[-1])
    else:
        predictions.append(output)

print("Calculating accuracy...")
targets = [example["answer"] for example in test_data]

em_score = exact_match.compute(predictions=predictions, references=targets, ignore_case=True, ignore_punctuation=True)["exact_match"]
print(f"Exact match : {em_score}")

predictions = [{
    "question": example["question"],
    "answer": example["answer"],
    "model_output": output,
    "prediction": pred
} for example, output, pred in zip(test_data, outputs, predictions)]


Calculating accuracy...
Exact match : 0.0


In [39]:
predictions

[{'question': 'The girls are trying to raise money for a carnival. Kim raises $320 more than Alexandra, who raises $430, and Maryam raises $400 more than Sarah, who raises $300. How much money, in dollars, did they all raise in total?',
  'answer': '2280',
  'model_output': 'trees planted today. 21 - 15 = 21. So the answer is 21. Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. There will be........... =.,...:.:..:.... So in total they had 32 + 42 = 74. After eating 35,',
  'prediction': '35'},
 {'question': 'Kalinda is working on a 360 piece puzzle with her mom. Kalinda can normally add 4 pieces per minute. Her mom can typically place half as many pieces per minute as Kalinda.  How many hours will it take them to complete this puzzle?',
  'answer': '1',
  'model_output': "trees planted today. are. ofs had 42.:'''' golf balls. + 2 = 7. So the answer is 7. Question: Shawn has 5 toys. He got 2 toys from his mom and dad. Now he has... golf ball

In [ ]:
# t5-Large, 0.02
# flan-t5-large, 0.03

In [ ]:

with open(os.path.join(args.save_dir, f"predictions.jsonl"), "w") as fout:
    for prediction in predictions:
        fout.write(json.dumps(prediction) + "\n") 

with open(os.path.join(args.save_dir, "metrics.json"), "w") as fout:
    json.dump({
        "exact_match": em_score
    }, fout, indent=4)
